In [1]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .

Processing /content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8829434 sha256=472d56d1a5f7c388650e83aeb8b1d7c1771b3d833fcca08e3adfb4b49fef9a62
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6xqx9fv/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


Cloning into 'transformers'...


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [4]:
!mkdir models/

In [5]:
import pandas as pd
import os
import numpy as np
import string

In [6]:
# Считаем тренировочные данные.

df_train = pd.read_csv('/content/drive/MyDrive/Диплом_2024/data/therapy_train_true.csv')

df_train.head()

,text
0,Московский государственный медико-стоматологич...
1,Башкирский Государственный Медицинский Универс...
2,Министерство здравоохранения Республики Белару...
3,\nПаспортная часть\n\nФИО: \nВозраст: 29 лет\n...
4,\nИстория болезни.\nФамилия: \n Имя: \nОтчест...


In [7]:
# Считаем тестовые данные.

df_test = pd.read_csv('/content/drive/MyDrive/Диплом_2024/data/therapy_test_true.csv')

df_test.head()

,text
0,\n\nМинистерство здравоохранения Российской Фе...


In [8]:
print(df_train.shape)

print(df_test.shape)

(65, 1)
(1, 1)


In [9]:
pd.set_option('display.max_colwidth', None)

df_train.loc[0]

text    Московский государственный медико-стоматологический университет\nкафедра пропедевтики внутренних болезней стоматологического факультета\n(заведующий кафедрой  - заслуженный деятель науки РФ, профессор Токмачев Юрий Константинович) \n\n\n\n\n\n\n\n\n\n\nИСТОРИЯ БОЛЕЗНИ\nБольного Коновалова А.Д., 65 лет\n5 терапевтическое отделение, палата 102\n\n\n\n\n\n\n\n\n\nКуратор студентка III курса\n 30 группы дневного \nстоматологического факультета\nКоваленко Александры Валериевны\n\n\n\nПреподаватель: Пихлак А.Э.\n\n\n\n\n\n\n\n\nМосква, 2000\n\n\n\nПаспортные данные\n\nФ.И.О.   Коновалов  Алексей Дмитриевич \nГод рождения 1935\nПрофессия:  на пенсии, ранее работал строителем, литейщиком пластмасс\nМесто жительства:  г. Москва, Бескуднический бульвар, 18-2-14\nДата поступления в клинику 13.11.00 \n\nЖалобы при поступлении\n\nна  внезапно возникшее чувство резкой нехватки воздуха, постепенно нараставшее и продолжавшееся до 30 минут, не проходящее при использовании ингалятора. В разгар п

In [10]:
# Уберем лишние знаки и цифры из текста трейн.

df_train = df_train.replace(r'\n', ' ', regex=True)

df_train = df_train.replace(r'\t', ' ', regex=True)

df_train['text'] = df_train['text'].str.lower()

df_train['text'] = df_train['text'].str\
                                   .replace('[{}]'.format(string.punctuation), '', regex=True)

df_train['text'] = df_train['text'].str.replace(' {2,}', ' ', regex=True)

from string import digits

remove_digits = str.maketrans('', '', digits)

df_train['text'] = df_train['text'].str.translate(remove_digits)


df_train.loc[0]

text    московский государственный медикостоматологический университет кафедра пропедевтики внутренних болезней стоматологического факультета заведующий кафедрой заслуженный деятель науки рф профессор токмачев юрий константинович история болезни больного коновалова ад  лет  терапевтическое отделение палата  куратор студентка iii курса  группы дневного стоматологического факультета коваленко александры валериевны преподаватель пихлак аэ москва  паспортные данные фио коновалов алексей дмитриевич год рождения  профессия на пенсии ранее работал строителем литейщиком пластмасс место жительства г москва бескуднический бульвар  дата поступления в клинику  жалобы при поступлении на внезапно возникшее чувство резкой нехватки воздуха постепенно нараставшее и продолжавшееся до  минут не проходящее при использовании ингалятора в разгар приступа появлялся кашель с отделением небольшого количества слизистой мокроты течение приступа облегчалось принятием сидячего положения с упором рук на колени прис

In [11]:
# Уберем лишние знаки и цифры из текста тест.

df_test = df_test.replace(r'\n', ' ', regex=True)

df_test = df_test.replace(r'\t', ' ', regex=True)

df_test['text'] = df_test['text'].str.lower()

df_test['text'] = df_test['text'].str\
                                 .replace('[{}]'.format(string.punctuation), '', regex=True)

df_train['text'] = df_train['text'].str.replace(' {2,}', ' ', regex=True)


remove_digits = str.maketrans('', '', digits)

df_test['text'] = df_test['text'].str.translate(remove_digits)


df_test

text
0    министерство здравоохранения российской федерации  алтайский государственный медицинский университет кафедра пропедевтики внутренних болезней зав кафедрой проф            академическая история болезни          больной куратор студентка  группы iii курса лечебного факультета время курации  –  г преподаватель           паспортная часть   фио   возраст  лет   место работы центр занятости населения  место жительства  дата поступления в клинику  г  диагноз пневмония в правой нижней доле дн ii острый бронхит pneumonia crouposae dextri lobi inferioris insufficientia respiratoria gradus ii                   status praesens subjectivus  жалобы больного на момент осмотра редкий сухой кашель одышка при физической нагрузке лёгкая слабость недомогание головокружение жалобы на момент поступления редкий сухой кашель одышка при физической нагрузке слабость недомогание распирающие головные боли головокружение шум в ушах закладывает уши при повышении и снижении ад боли за грудиной давящего характера   anamnesis morbi  считает себя больным с  г когда появился редкий сухой кашель одышка при физической нагрузке головные боли головокружение к врачу обращаться не стал самостоятельно ни чем не лечился заболевание не прогрессировало состояние больного было удовлетворительным  изза выраженных головных болей вызвал скорую помощь и был госпитализирован в городскую больницу № с диагнозом гипертонический криз на момент поступления ад  при рабочем  температура   anamnesis vitae  родился  г в деревне в многодетной семье в детском возрасте переехал с родителями в город младший ребёнок в семье имеет пять братьев и три сестры физически и интеллектуально развивался нормально от сверстников не отставал с ми лет пошел в школу учился хорошо в период с  г по  гпроходил службу в рядах советской армии в  г сбежал из армии женился и устроился работать в гаи после некоторое время работал в сфере связи монтёром окончил оренбургский железнодорожный техникум получил профессию машиниста и в течение ти лет работал по профессии затем в течение пяти лет электромонтёром на атс где как утверждает работал с кислотами в данное время не работает и состоит на учёте в центре занятости  семейный анамнез женат имеет двух взрослых детей и двух внуков наследственность   легенда пробанд – ii отец пробанда – i умер естественной смертью в старческом возрасте причина неизвестна мать пробанда – i умерла естественной смертью в старческом возрасте причина неизвестна братья пробанда – ii здоровы сестры пробанда – ii здоровы жена пробанда – ii здорова сын пробанда – iii здоров дочь пробанда – iii здорова внуки пробанда – iv здоровы наследственные и онкологические заболевания у родителей не знает у родственников отрицает  бытовой анамнез материально обеспечен проживает с женой в х комнатной квартире питание регулярное  раза в день полноценное разнообразное эпидемиологический анамнез инфекционный гепатит венерические заболевания тифы малярию и туберкулез отрицает за последние шесть месяцев кровь не переливалась у стоматолога не лечился инъекции не производились за пределы города не выезжал и контакта с инфекционными больными не имел последняя флг в мае  года эмоциональнонервнопсихический анамнез пережил тяжёлое психоэмоциональное переживание в  г когда попал в автомобильную аварию в результате которого произошёл инфаркт миокарда перенесенные заболевания в детстве несколько раз болел орз сведениями о перенесённых детских заболеваниях не располагает в  году была произведена аппендэктомия оим в  году травм не было хронические заболевания  стенокардия напряжения iii функционального класса в течение х лет  принимает сустак  таблетку  раз в день приступы загрудинных болей снимает приёмом нитроглицерина  гипертоническая болезнь с  г ад повышается до  мм рт ст принимает капотен  мг  раза в день  привычные интоксикации не курит алкоголь употребляет умеренно крепким чаем кофе не злоупотребляет употребление наркотических веществ отрицает гемотрансфузионный анамнез гемотрансфузии раньше не производил

In [12]:
with open('train.txt', 'a') as f:
    df_string = df_train.to_string(header=False, index=False)
    f.write(df_string)

In [13]:
with open('valid.txt', 'a') as f:
    df_string = df_test.to_string(header=False, index=False)
    f.write(df_string)

In [14]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

--2024-04-10 23:20:33--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28908 (28K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  28.23K  --.-KB/s    in 0.001s  

2024-04-10 23:20:33 (19.0 MB/s) - ‘run_clm.py’ saved [28908/28908]



In [15]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [16]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train.txt \
    --validation_file valid.txt \
    --num_train_epochs 30\
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --block_size 64 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --save_total_limit 1\
    --output_dir models/therapy_gpt

2024-04-10 23:22:00.135303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 23:22:00.135368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 23:22:00.249459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 23:22:02.273352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/10/2024 23:22:09 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/10/2024 23:22:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
ac

In [17]:
import numpy as np
import torch

In [18]:
np.random.seed(42)
torch.manual_seed(42)

In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [21]:
tok = GPT2Tokenizer.from_pretrained("models/therapy_gpt")

In [22]:
model = GPT2LMHeadModel.from_pretrained("models/therapy_gpt")

In [23]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [24]:
text = "хрипы в"
inpt = tok.encode(text, return_tensors="pt")

In [25]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [26]:
out

tensor([[  292,   312,  1923,   282, 17203, 32751,  3236,  9492, 33607, 33204]],
       device='cuda:0')

In [27]:
tok.decode(out[0])

'хрипы в легких выслушиваются рассеянные сухие'

In [28]:
text = "жалобы"
inpt = tok.encode(text, return_tensors="pt")

In [29]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [30]:
out

tensor([[ 1646,   627, 19143,   322, 11103,  3576,  5296, 14001,  3380,   539]],
       device='cuda:0')

In [31]:
tok.decode(out[0])

'жалобы больной не предъявляет состояние удовлетворительное'

In [32]:
text = "бронхиальная"
inpt = tok.encode(text, return_tensors="pt")

In [33]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [34]:
out

tensor([[  296,  1368,  1711,  3050, 37847,   429,  2119,   669,   549,  5189]],
       device='cuda:0')

In [35]:
tok.decode(out[0])

'бронхиальная астма смешаная форма'

In [36]:
text = "пациент"
inpt = tok.encode(text, return_tensors="pt")

In [37]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [41]:
out

tensor([[  513,   377,   645,   343, 10050, 13029,  1979,  7925,   371,  5744]],
       device='cuda:0')

In [40]:
tok.decode(out[0])

'пациентка отмечает ухудшение состояния при нах'

In [19]:
!zip -r /content/gpt_2_3_therapy.zip /content/models/therapy_gpt

  adding: content/models/therapy_gpt/ (stored 0%)
  adding: content/models/therapy_gpt/merges.txt (deflated 76%)
  adding: content/models/therapy_gpt/all_results.json (deflated 54%)
  adding: content/models/therapy_gpt/eval_results.json (deflated 41%)
  adding: content/models/therapy_gpt/vocab.json (deflated 74%)
  adding: content/models/therapy_gpt/tokenizer_config.json (deflated 75%)
  adding: content/models/therapy_gpt/training_args.bin (deflated 52%)
  adding: content/models/therapy_gpt/special_tokens_map.json (deflated 80%)
  adding: content/models/therapy_gpt/runs/ (stored 0%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/ (stored 0%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/events.out.tfevents.1712795456.b01ea1c613e5.1915.1 (deflated 26%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/events.out.tfevents.1712791343.b01ea1c613e5.1915.0 (deflated 63%)
  adding: content/models/therapy_gpt/tokenizer.json (de